In [2]:
#!/usr/bin/env python3
import pyreadstat
import sys
import chardet
import pandas as pd
import os

def export_csv_to_sav(path):
    
    
   

    #print(f"Chargement du fichier CSV depuis : {path}")
    
        
    try:
        # Détecter l'encodage du fichier CSV
        with open(path, 'rb') as f:
            result = chardet.detect(f.read())

        # Lire le fichier CSV avec l'encodage détecté
        df_originall = pd.read_csv(path, encoding=result['encoding'])
        
        # Définir le chemin de sortie pour le fichier SAV
        sav_path = path.replace('.csv', '.sav')

        # Créer un dictionnaire pour stocker les métadonnées
        metadata_dict = {}
        for _, row in df_originall.iterrows():
            question_id = row['question_id']
            option_index = row['option_index']
            response_text = row.get('response_text', None)  # Supposons qu'il y a une colonne 'response_text'
            answer = row.get('answer', None)  # Supposons qu'il y a une colonne 'answer'

            # Si 'question_id' n'est pas déjà dans le dictionnaire, l'ajouter
            if question_id not in metadata_dict:
                metadata_dict[question_id] = {
                    'option_indexs': [],  # Liste pour stocker les index d'option
                    'response_texts': [],  # Liste pour stocker les textes de réponse
                    'answers': []  # Liste pour stocker les réponses
                }
            
            # Ajouter les données associées à l'option (si elles existent)
            if not pd.isna(option_index):
                metadata_dict[question_id]['option_indexs'].append(option_index)
            
            # Ajouter le texte de réponse et la réponse si disponibles
            metadata_dict[question_id]['response_texts'].append(response_text)
            metadata_dict[question_id]['answers'].append(answer)

        # Créer un dictionnaire pour stocker les textes des questions
        question_text_dict = pd.Series(df_originall['question_text'].values, index=df_originall['question_id']).to_dict()

        # Exclure la colonne 'question_text' pour les transformations
        df_original = df_originall.drop(columns=['question_text'])

        # Séparer la colonne 'date' avant de pivoter
        date_column = df_original[['session_id', 'date', 'comment', 'longitude']].drop_duplicates()  # Une seule ligne par session

        # Ajouter un suffixe aux colonnes 'option_index' et 'response_text'
        df_original['variable'] = df_original.groupby(['session_id', 'question_id']).cumcount().astype(str)

        # Pivoter les données en excluant la colonne 'date'
        df_transformed = df_original.pivot(index='session_id', columns=['question_id', 'variable'], values=['option_index', 'response_text', 'answer'])

        # Aplatir les niveaux de colonnes
        df_transformed.columns = [f'{col[1]}_{col[0]}' for col in df_transformed.columns]

        # Réinitialiser l'index
        df_transformed = df_transformed.reset_index()

        # Réassocier la colonne 'date' à la bonne session après le pivot
        df_transformed = pd.merge(df_transformed, date_column, on='session_id', how='left')

        # Renommer les colonnes selon le format final en utilisant un dictionnaire
        column_rename_dict = {}
        for col in df_transformed.columns:
            if len(col) == 2:  # S'assurer que la colonne a deux parties
                question_id = col[0].split('_')[0]
                column_type = col[0].split('_')[1] if '_' in col[0] else ''  # soit 'option_index' ou 'response_text'
                new_column_name = f'{question_id}_{col[1]}_{column_type}'
                column_rename_dict[col] = new_column_name

        # Renommer les colonnes selon le dictionnaire créé
        df_transformed = df_transformed.rename(columns=column_rename_dict)

        # Enregistrer le DataFrame transformé en CSV
        df_transformed.to_csv(path, index=False, encoding='utf-8')

        #=============================================== STEP 2 : transformé les titres avec Q pour le sav


        # Détecter l'encodage du fichier CSV
        with open(path, 'rb') as f:
            result = chardet.detect(f.read())

        # Utiliser l'encodage détecté pour lire le fichier CSV
        df_original = pd.read_csv(path, encoding=result['encoding'])

        # Créer des dictionnaires pour stocker les correspondances entre les indices et les lettres
        option_index_dict = {}
        response_text_dict = {}
        answer_dict = {}

        # Liste pour stocker les questions à multiple options
        multiple_option_questions = {}

        # Renommer les colonnes selon le format final en utilisant un dictionnaire
        column_rename_dict = {}
        for col in df_original.columns:
            if 'option_index' in col:
                question_id, index = col.split('_')[0], df_original[col].iloc[0]
                new_col = f'Q_{question_id}_O_{index}'
                option_index_dict[col] = new_col  # Stocker l'ancienne et la nouvelle colonne dans le dictionnaire
                
                # Ajouter la question à multiple_option_questions s'il y a plusieurs "O" pour un même question_id
                if question_id not in multiple_option_questions:
                    multiple_option_questions[question_id] = set()  # Utiliser un set pour éviter les doublons
                multiple_option_questions[question_id].add(new_col)  # Ajouter les options pour cette question

            elif 'response_text' in col:
                question_id, index = col.split('_')[0], df_original[col.replace('response_text', 'option_index')].iloc[0]
                new_col = f'Q_{question_id}_R_{index}'
                response_text_dict[col] = new_col  # Stocker l'ancienne et la nouvelle colonne dans le dictionnaire
            elif 'answer' in col:  # Ajouter la logique pour renommer les colonnes "answer"
                question_id, index = col.split('_')[0], df_original[col.replace('answer', 'option_index')].iloc[0]
                new_col = f'Q_{question_id}_A_{index}'
                answer_dict[col] = new_col  # Stocker l'ancienne et la nouvelle colonne dans le dictionnaire
            else:
                new_col = col  # Si la colonne ne correspond pas à l'un des types ci-dessus, on ne change pas le nom

            column_rename_dict[col] = new_col

        # Filtrer les questions qui ont plusieurs options (plus d'une colonne O associée)
        multiple_option_questions = {q_id: options for q_id, options in multiple_option_questions.items() if len(options) > 1}

        # Utiliser les dictionnaires pour renommer les colonnes "option_index", "response_text" et "answer"
        df_original = df_original.rename(columns={**option_index_dict, **response_text_dict, **answer_dict})

        # Utiliser le dictionnaire pour renommer toutes les colonnes selon le format final
        df_original = df_original.rename(columns=column_rename_dict)

        # Enregistrer le DataFrame transformé en CSV
        df_original.to_csv(path, index=False, encoding='utf-8')

        #============================================== STEP 3 : Organier les colonnes question avec son option et réponse

        # Charger le fichier CSV transformé
        df = pd.read_csv(path)

        # Extraire les noms de colonnes liés aux option_indexs (O) et aux réponses (R) et answer (A)
        option_columns = [col for col in df.columns if 'O_' in col]
        #response_columns = [col for col in df.columns if 'R_' in col]
        #answer_columns = [col for col in df.columns if 'A_' in col]

        # Organiser les colonnes de O suivies de leurs colonnes de R pour chaque question
        new_columns_order = []
        for option_col in option_columns:
            question_id = option_col.split('_')[1]
            response_col = option_col.replace('O_', 'R_')
            answer_col = option_col.replace('O_', 'A_')
            
            # Ajouter les colonnes O, R, et A pour chaque question dans le bon ordre
            new_columns_order.extend([option_col, response_col, answer_col])

        # Créer un nouveau DataFrame réorganisé
        df_reorganized = df[['session_id'] + ['date'] + ['comment'] + ['longitude']+new_columns_order]

        # Enregistrer le DataFrame réorganisé en CSV
        df_reorganized.to_csv(path, index=False, encoding='utf-8')

        #============================================== STEP 4 : distinguer les questions à champ libre et faire en sorte a avoir des float sur les titres

      
        

        # Charger le fichier CSV réorganisé
        df_reorganized = pd.read_csv(path, encoding='utf-8')

        # Convertir la colonne date en type datetime
        df_reorganized['date'] = pd.to_datetime(df_reorganized['date'])

        # Ouvrir le fichier CSV en mode binaire pour détecter l'encodage
        with open(path, 'rb') as f:
            result = chardet.detect(f.read())

        # Charger le fichier CSV avec l'encodage détecté
        #df_csv = pd.read_csv(path, encoding=result['encoding'])

        # Convertir option_index en entier si possible
        #df_csv['option_index'] = pd.to_numeric(df_csv['option_index'], errors='coerce').dropna().astype('Int64')

        # 1. Lister les questions à champ libre (où l'option_index est NaN)
        questions_reponse_libre = [q_id for q_id, data in metadata_dict.items() if not data['option_indexs']]

        # 2. Parcourir chaque question à réponse libre et effectuer les modifications nécessaires
        for question_id in questions_reponse_libre:
            # 2.1. Renommer la colonne de la question libre en supprimant 'O_' et tout ce qui suit
            old_column_name_prefix = f"Q_{question_id}_O_"
            
            # Rechercher les colonnes qui correspondent à ce modèle
            matching_columns = [col for col in df_reorganized.columns if col.startswith(old_column_name_prefix)]
            
            for col in matching_columns:
                # 2.2. Remplacer le titre en supprimant 'O_' et tout ce qui suit
                new_column_name = col.split('_O_')[0]  # Conserver uniquement la partie avant '_O_'
                df_reorganized.rename(columns={col: new_column_name}, inplace=True)
                
                # 3. Remplir la colonne renommée avec les données de la colonne correspondante 'R_'
                r_column = col.replace('O_', 'R_')
                if r_column in df_reorganized.columns:
                    df_reorganized[new_column_name] = df_reorganized[r_column]
                
                # 4. Supprimer la colonne 'R_' et 'A_' correspondantes
                a_column = col.replace('O_', 'A_')
                if r_column in df_reorganized.columns:
                    df_reorganized.drop(columns=[r_column], inplace=True)
                if a_column in df_reorganized.columns:
                    df_reorganized.drop(columns=[a_column], inplace=True)

        # Assurer que les noms de colonnes avec 'O_' aient des entiers dans leur nom, et non des floats
        # Liste des suffixes à traiter
        suffixes = ['O', 'A', 'R']

        for suffix in suffixes:
            for col in df_reorganized.columns:
                if f'_{suffix}_' in col:
                    parts = col.split(f'_{suffix}_')
                    if len(parts) > 1:
                        try:
                            # Convertir l'option_index en entier
                            option_index = int(float(parts[1]))
                            new_col_name = f"{parts[0]}_{suffix}_{option_index}"
                            df_reorganized.rename(columns={col: new_col_name}, inplace=True)
                        except ValueError:
                            pass  # Si l'option_index n'est pas un nombre valide, on l'ignore

        # Écrire le fichier SAV après les modifications
        pyreadstat.write_sav(df_reorganized, sav_path)

        #============================================== STEP 5 : création des values labels en supprimant les colonnes Response et Answer

        # Charger le fichier SAV et ses métadonnées
        df_sav, meta = pyreadstat.read_sav(sav_path)

        # Transformer la colonne 'date' au format datetime si nécessaire
        df_sav['date'] = pd.to_datetime(df_sav['date'])

        # Créer un dictionnaire pour stocker les value labels
        value_labels_dict = {}

        # Ouvrir le fichier CSV en mode binaire pour détecter l'encodage
        with open(path, 'rb') as f:
            result = chardet.detect(f.read())

        # Charger le fichier CSV avec l'encodage détecté
        #df_csv = pd.read_csv(path, encoding=result['encoding'])

        #----
        # Itérer sur chaque question dans le dictionnaire metadata_dict
        for question_id, data in metadata_dict.items():
            option_indexs = data['option_indexs']  # Liste des option_indexs
            response_texts = data['response_texts']  # Liste des response_texts associés

            # Assurez-vous qu'il n'y a pas de doublons dans 'option_index' et 'response_texts'
            unique_option_indexs = list(dict.fromkeys(option_indexs))
            unique_response_texts = list(dict.fromkeys(response_texts))

            # Créer le dictionnaire des value labels en associant option_index à response_text
            value_labels_dict[question_id] = dict(zip(unique_option_indexs, unique_response_texts))
        #----

        # S'assurer que les variable_value_labels existent dans les métadonnées
        if meta.variable_value_labels is None:
            meta.variable_value_labels = {}

        # Ajouter les value labels aux métadonnées
        for question_id, value_dict in value_labels_dict.items():
            for option_index, response_text in value_dict.items():
                # Convertir option_index en entier
                option_index_int = int(float(option_index))  # Gérer le cas où option_index est un flottant
                var_name = f"Q_{question_id}_O_{option_index_int}"

                # Vérifier si la question est à choix unique ou multiple
                if f"Q_{question_id}_O_" in var_name and not var_name.endswith('.0.1'):  # Choix unique : Une seule colonne 'O'
                    
                    # S'assurer que la colonne est bien présente dans les métadonnées
                    if var_name not in meta.variable_value_labels:
                        meta.variable_value_labels[var_name] = {}
                    
                    # Remplacer O par NaN si A contient NaN
                
                    if f"Q_{question_id}_A_{option_index_int}" in df_sav.columns :
                        df_sav.loc[pd.isna(df_sav[f"Q_{question_id}_A_{option_index_int}"]), var_name] = pd.NA

                    # Ajouter le label pour l'option choisie
                    meta.variable_value_labels[var_name][option_index_int] = response_text
                    
                
                # Ajouter le value label pour chaque option_index qui existe 
                for option_index_int, response_text in value_dict.items():
                    if pd.notna(option_index_int):
                        meta.variable_value_labels[var_name][option_index_int] = response_text

        #-----
        # Vérifiez que vous parcourez les bonnes clés
        for question_id, data in metadata_dict.items():
            # Imprimer le type de question_id pour déboguer
            question_id_str = str(question_id)
            
            # Vérifier si question_id est une chaîne de caractères
            if question_id_str in multiple_option_questions:
                # Identifier la dernière option index de la question
                dernier_option_index = max(data['option_indexs']) # Prendre le dernier option_index
                
                # Construire la variable correspondant à cette dernière option
                dernier_option_var = f"Q_{question_id}_O_{dernier_option_index}.1"

                # Vérifier si cette colonne existe dans le DataFrame
                if dernier_option_var in df_sav.columns:
                    # Renommer la colonne en fonction du format S_Q_{question_id}_{dernier_option_index}
                    dernier_option_index_int = int(float(dernier_option_index))
                    new_column_name = f"S_Q_{question_id}_{dernier_option_index_int}"
                    df_sav.rename(columns={dernier_option_var: new_column_name}, inplace=True)

                    # Remplacer le contenu de la nouvelle colonne avec celui de la colonne 'R_' correspondante
                    r_column = f"Q_{question_id}_R_{dernier_option_index}.1"
                    if r_column in df_sav.columns:
                        df_sav[new_column_name] = df_sav[r_column]

        #-----
        # Supprimer les colonnes A et R
        columns_to_drop = [col for col in df_sav.columns if '_A_' in col or '_R_' in col]
        df_sav = df_sav.drop(columns=columns_to_drop)

        #-----
        # Vérifier si meta.column_labels est une liste
        if isinstance(meta.column_labels, list):
            # Créer un dictionnaire en associant chaque colonne à son libellé ou une chaîne vide si pas de libellé
            meta.column_labels = {col: "" for col in df_sav.columns}

        # Mettre à jour les labels des colonnes
        meta.column_labels = {col: label for col, label in meta.column_labels.items() if col in df_sav.columns}

        # Réécrire le fichier SAV avec les value labels mis à jour
        pyreadstat.write_sav(df_sav, 
                            sav_path, 
                            file_label="Updated SAV with Corrected Value Labels",
                            variable_value_labels=meta.variable_value_labels,
                            column_labels=meta.column_labels)

        #============================================== STEP 6 : modifier les titres en Q_idquestion pour les questions à choix unique

        # Charger le fichier SAV avec les value labels
        df_sav, meta = pyreadstat.read_sav(sav_path)

        # Identifier les colonnes à renommer
        column_mapping = {}

        for col in df_sav.columns:
            if col.startswith('Q_') and '_O_' in col:
                question_id = col.split('_')[1]  # Extraire l'ID de la question
                if question_id not in column_mapping:
                    column_mapping[question_id] = []
                column_mapping[question_id].append(col)

        # Créer un dictionnaire de renommage pour les colonnes à choix unique
        rename_dict = {}
        for question_id, columns in column_mapping.items():
            if len(columns) == 1:  # S'il n'y a qu'une seule colonne pour cette question
                original_column = columns[0]
                new_column_name = f'Q_{question_id}'  # Renommer vers Q_1, Q_2, etc.
                rename_dict[original_column] = new_column_name

        # Conserver les value labels avant de renommer
        value_labels_backup = {col: meta.variable_value_labels.get(col, {}) for col in rename_dict.keys()}

        # Renommer les colonnes dans le DataFrame
        df_sav.rename(columns=rename_dict, inplace=True)

        # Réappliquer les value labels aux nouvelles colonnes renommées
        for original_col, new_col in rename_dict.items():
            if new_col not in meta.variable_value_labels:
                meta.variable_value_labels[new_col] = value_labels_backup[original_col]

        # Réécrire le fichier SAV avec les colonnes renommées et les value labels
        pyreadstat.write_sav(df_sav, 
                            sav_path, 
                            file_label="Updated SAV with Unique Column Titles",
                            variable_value_labels=meta.variable_value_labels,
                            column_labels=meta.column_labels)  # Conserver les labels des colonnes

        #print("Les colonnes ont été renommées tout en préservant les value labels.")
        #============================================== STEP 7 : création des libellés en gardant les value labels

        # Charger le fichier SAV avec les value labels déjà présents
        df_sav, meta = pyreadstat.read_sav(sav_path)

        # Déterminer le maximum d'index d'options à partir des noms de colonnes
        max_option_index = 0
        for col in df_sav.columns:
            if col.startswith("Q_"):  # S'assurer qu'il s'agit d'une colonne de question
                # Extraire l'index d'option de la colonne, par exemple Q_1_O_3.1
                parts = col.split("_")
                if len(parts) > 3 and parts[2] == "O":
                    option_index = int(parts[3])  # Convertir l'index d'option en entier
                    max_option_index = max(max_option_index, option_index)

        # Si metadata.column_labels est une liste, la convertir en dictionnaire
        if isinstance(meta.column_labels, list):
            meta.column_labels = {col: "" for col in df_sav.columns}

        # Créer un dictionnaire pour les libellés de colonnes
        column_labels = {}

        # Ajouter les libellés pour les colonnes de questions à choix unique
        for question_id in question_text_dict:
            # Libellé pour la colonne Q_idquestion
            q_col_name = f"Q_{question_id}"
            if q_col_name in df_sav.columns:
                column_labels[q_col_name] = question_text_dict[question_id]  # Utiliser le texte de la question

        # Ajouter les libellés pour les colonnes de questions à options multiples
        for question_id in question_text_dict:
            for option_index in range(1, max_option_index + 1):  # Utiliser le max_option_index trouvé
                # Libellé pour les colonnes Q_idquestion_O_optionindex
                q_col_name = f"Q_{question_id}_O_{option_index}"
                if q_col_name in df_sav.columns:
                    column_labels[q_col_name] = question_text_dict[question_id]  # Utiliser le texte de la question

                # Libellé pour les colonnes S_Q_idquestion_optionindex
                s_col_name = f"S_Q_{question_id}_{option_index}"
                if s_col_name in df_sav.columns:
                    column_labels[s_col_name] = f"{question_text_dict[question_id]} Autres"  # Texte de la question + 'Autres'

        # Ajouter les libellés pour les autres colonnes (nom de colonne)
        for col in df_sav.columns:
            if col not in column_labels:
                column_labels[col] = col  # Utiliser le nom de la colonne comme libellé

        # Mettre à jour les column_labels
        meta.column_labels.update(column_labels)

        # Réécrire le fichier SAV avec les column labels mis à jour
        pyreadstat.write_sav(df_sav, sav_path, 
                            column_labels=meta.column_labels, 
                            variable_value_labels=meta.variable_value_labels)
        #============================================== STEP 8 : trouver les measures

        

        # Charger le fichier SAV avec ses métadonnées
        df, meta = pyreadstat.read_sav(sav_path)

        # Fonction pour déterminer le type de measure
        def set_measure(df, meta):
            # On récupère les informations actuelles
            variable_measure = meta.variable_measure.copy()  # Copier pour éviter de modifier directement l'original
            variable_value_labels = meta.variable_value_labels  # Pour vérifier si une colonne a des value labels
            
            for col in df.columns:
                # 1. Ordinal: Si la colonne a des value labels (multiple choix ou unique avec options) ou de type date
                if col in variable_value_labels or df[col].dtype == 'datetime64[ns]':
                    variable_measure[col] = "ordinal"
                
                # 2. Échelle (Scale): Si c'est un champ libre avec réponse numérique sans value labels
                elif df[col].dtype == 'float64' or df[col].dtype == 'int64':
                    variable_measure[col] = "scale"
                
                # 3. Nominal: Si c'est un champ libre texte sans value labels
                elif df[col].dtype == 'object':
                    variable_measure[col] = "nominal"

            return variable_measure

        # Mettre à jour les measures
        updated_variable_measure = set_measure(df, meta)

        # Sauvegarder le fichier avec les mesures mises à jour tout en gardant les anciennes métadonnées
        pyreadstat.write_sav(
            df,
            sav_path,
            variable_value_labels=meta.variable_value_labels,
            column_labels=meta.column_labels,
            variable_measure=updated_variable_measure
        )

        #============================================== STEP 9 : création des -1 pour NaN et l'ajouter dans manquants des vues des variables
        
        # Charger le fichier SAV avec les value labels déjà présents
        df_sav, meta = pyreadstat.read_sav(sav_path)

        # Créer un dictionnaire pour stocker les plages de valeurs manquantes (missing_ranges)
        missing_ranges = {}

        # Itérer sur les colonnes pour remplir les valeurs manquantes dans la vue des variables
        for col in df_sav.columns:
            # Vérifier si la colonne correspond à une question Q et si elle est numérique
            if col.startswith('Q') and pd.api.types.is_numeric_dtype(df_sav[col]):
                
                # Remplir avec -1 si la valeur est NaN dans la vue des données
                df_sav[col] = df_sav[col].fillna(-1)

                # Convertir en entier
                df_sav[col] = df_sav[col].astype(int)
                
                # Définir -1 comme valeur manquante dans la vue des variables pour cette colonne
                missing_ranges[col] = [-1]  # Plage de valeurs manquantes fixée à -1

        # Vérifier et corriger les value labels pour s'assurer qu'ils sont tous des entiers
        for col in meta.variable_value_labels.keys():
            if col in df_sav.columns:
                
                # Assurez-vous que les labels de valeur soient des entiers
                meta.variable_value_labels[col] = {int(k): v for k, v in meta.variable_value_labels[col].items()}

                
        # Enregistrer le DataFrame mis à jour dans un nouveau fichier SAV avec les valeurs manquantes dans la vue des variables
        pyreadstat.write_sav(df_sav, sav_path, 
                            variable_value_labels=meta.variable_value_labels,
                            column_labels=meta.column_labels,
                            variable_measure=meta.variable_measure,
                            missing_ranges=missing_ranges)  # Ajouter les valeurs manquantes dans la vue des variables
        #supprimer le csv
        os.remove(path)

    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# Exécution de la fonction avec le chemin passé en argument
if __name__ == "__main__":
    if len(sys.argv) > 1:
        export_csv_to_sav(sys.argv[1])
    else:
        print("Veuillez fournir un chemin vers le fichier CSV.")




Une erreur s'est produite : [Errno 2] No such file or directory: '--ip=127.0.0.1'


In [3]:
export_csv_to_sav('/Users/Lisa/Desktop/response.csv')